# Convolutional Neural Network (CNN, ConvNet)

Neuronové sítě pro rozpoznávání a zpracovávání obrazu. Typická architektura:

![CNN Architecture](img/cnn_architecture.jpg)

Konvoluční vrstvy aplikují filtr (konvoluci), která umožňuje detekci elementární geometrických tvarů (například čáry či křivky různých orientací):

![CNN First Layer](img/cnn_first_layer.jpg)

Další vrstvy postupně kombinují základní tvary a jsou schopné detekovat složitější a složitější struktury:

![CNN Other Layers](img/cnn_second_layer.jpg)

Kód níže ukazuje, jak takový konvoluční filtr může fungovat.

In [ ]:
# import of common python libraries
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 6, 3
import seaborn as sns

In [ ]:
plt.rcParams['axes.grid'] = False
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False
plt.rcParams['xtick.labelsize'] = 0
plt.rcParams['ytick.labelsize'] = 0
plt.rcParams['axes.spines.top'] = True
plt.rcParams['axes.spines.right'] = True

In [ ]:
# loading of MNIST data stored on the drive
X_raw = np.load('data\\mnist_x.npy')
X = (X_raw / 255).astype('float32').T

In [ ]:
# show one image
n = 2
plt.imshow(X[:,n].reshape(28, 28), cmap=plt.cm.gray_r, interpolation='nearest', vmin=0, vmax=1)

Filtr o velikosti 5x5 pro detekci svislých čar může být definovaný například takto (v praxi se neuronová síť naučí rozpoznávat základní typy geometrických tvarů sama o sobě). Neurony v konvoluční vrstvě jsou aktivovány tam, kde se vyskytují příslušné vzory na obrázku.

In [ ]:
vertical_filter = np.array([
    [-1, 1, 1, -1, 0],
    [-1, 1, 1, -1, 0],
    [-1, 1, 1, -1, 0],
    [-1, 1, 1, -1, 0],
    [0, 0, 0, 0, 0]    
])

In [ ]:
plt.imshow(vertical_filter, cmap=plt.cm.RdBu, interpolation='nearest', vmin=-1.5, vmax=1.5)

In [ ]:
from scipy.signal import convolve2d

In [ ]:
black = np.ones(25)
white = np.zeros(25)
vertical = np.array([
    0, 1, 1, 0, 0,
    0, 1, 1, 0, 0, 
    0, 1, 1, 0, 0,
    0, 1, 1, 0, 0, 
    0, 1, 1, 0, 0,
])
horizontal = np.array([
    0, 0, 0, 0, 0,
    1, 1, 1, 1, 1,
    1, 1, 1, 1, 1,
    0, 0, 0, 0, 0,
    0, 0, 0, 0, 0,
])

In [ ]:
small_images = [black, white, vertical, horizontal]

In [ ]:
plt.rcParams['figure.figsize'] = 12, 4
fig, ax = plt.subplots(1, 4)
for i, img in enumerate(small_images):
    ax[i].imshow(img.reshape([5, 5]), cmap=plt.cm.gray_r, interpolation='nearest', vmin=0, vmax=1)
plt.rcParams['figure.figsize'] = 6, 3

In [ ]:
plt.imshow(vertical_filter, cmap=plt.cm.RdBu, interpolation='nearest', vmin=-1.5, vmax=1.5)

Aplikace filtru znamená překrytí filtru a zdrojového obrázku a následné pronásobení a nasčítání všech hodnot (tato operace se matematicky nazývá konvoluce). Pokud je zdrojový obrázek větší než filtr, pak se filtr postupně posouvá po celém obrázku a vytvoří se výstupní matice s aplikovaným filtrem, která ukazuje, kde byl daný geometrický tvar detekovaný.

In [ ]:
[convolve2d(img.reshape(5, 5), vertical_filter[::-1,::-1], mode='valid') for img in small_images]

In [ ]:
conv = convolve2d(X[:,n].reshape(28, 28), vertical_filter[::-1,::-1], mode='valid')

In [ ]:
def relu(x):
    return np.maximum(0, x)

In [ ]:
xs = np.linspace(-4, 4, 101)
ys = relu(xs)
sns.lineplot(xs, ys, label='relu')

In [ ]:
plt.imshow(X[:,n].reshape(28, 28), cmap=plt.cm.gray_r, interpolation='nearest', vmin=0, vmax=1)

In [ ]:
plt.imshow(relu(conv), cmap=plt.cm.gray_r, interpolation='nearest')

In [ ]:
horizontal_filter = vertical_filter.T
major_diag_filter = np.array([
    [0, 1, -1, 0, 0],
    [-1, 1, 1, -1, 0],
    [0, -1, 1, 1, -1],
    [0, 0, -1, 1, 1],
    [0, 0, 0, -1, 0]
])
minor_diag_filter = major_diag_filter[:, ::-1]

In [ ]:
labelled_filters = [
    ('Vertical', vertical_filter),
    ('Horizontal', horizontal_filter),
    ('Major diag', major_diag_filter),
    ('Minor diag', minor_diag_filter)
]

In [ ]:
plt.rcParams['figure.figsize'] = 12, 4
fig, ax = plt.subplots(1, 4)
for i, (l, f) in enumerate(labelled_filters):
    ax[i].imshow(f, cmap=plt.cm.RdBu, interpolation='nearest', vmin=-1.5, vmax=1.5)
    ax[i].set_title(l)
plt.rcParams['figure.figsize'] = 6, 3

In [ ]:
im = X[:, np.random.randint(X.shape[1])].reshape(28, 28)
plt.imshow(im, cmap=plt.cm.gray_r, interpolation='nearest', vmin=0, vmax=1)

In [ ]:
plt.rcParams['figure.figsize'] = 12, 4
fig, ax = plt.subplots(1, 4)

for j, (l, f) in enumerate(labelled_filters):
    vert_conv = relu(convolve2d(im, f[::-1,::-1], mode='valid'))
    ax[j].imshow(vert_conv, cmap=plt.cm.gray_r, interpolation='nearest', vmin=0, vmax=12)
    ax[j].set_title(l)

plt.rcParams['figure.figsize'] = 6, 3

In [ ]:
plt.rcParams['figure.figsize'] = 16, 16

fig, ax = plt.subplots(5, 5)
plt.subplots_adjust(hspace=0.3)
perm = np.random.permutation(X.shape[1])[:5]
for i, n in enumerate(perm):
    im = X[:,n].reshape(28, 28)
    ax[i][0].imshow(im, cmap=plt.cm.gray_r, interpolation='nearest', vmin=0, vmax=1)
    ax[i][0].set_title('Number')
    
    for j, (l, f) in enumerate(labelled_filters):
        vert_conv = relu(convolve2d(im, f[::-1,::-1], mode='valid'))
        ax[i][1 + j].imshow(vert_conv, cmap=plt.cm.gray_r, interpolation='nearest', vmin=0, vmax=12)
        ax[i][1 + j].set_title(l)
    
plt.rcParams['figure.figsize'] = 6, 3

Konvoluční vrstva je typicky kombinována s tzv. pooling vrstvou - zmenší rozměry obrázku na polovinu, zpravidla nahradí plošku 2x2 průměrem či maximem z daných hodnot. Konvoluční neuronová síť se sama naučí vhodné filtry podle trénovacích vstupních dat.